# Chapter 5 - Ensemble machine learning, deep learning

2022 February 23

![kandc](img/kandc.jpg)

[Texas Monthly, Music Monday: Uncovering The Mystery Of The King & Carter Jazzing Orchestra](https://www.texasmonthly.com/the-daily-post/music-monday-uncovering-the-mystery-of-the-king-carter-jazzing-orchestra/)

## Ensemble machine learning

"Ensemble machine learning methods use multiple learning algorithms to obtain better predictive performance than could be obtained from any of the constituent learning algorithms." [H2O.ai ensemble example](https://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/stacked-ensembles.html)

In this manner, stacking/SuperLearner ensembles are powerful tools because they: 

1. Eliminates bias of single algorithm selection for framing a research problem.

2. Allows for comparison of multiple algorithms, and/or comparison of the same model but tuned in many different ways.

3. Utilizes a second-level algorithm that produces an ideal weighted prediction that is suitable for data of virtually all distributions and uses external cross-validation to prevent overfitting.

The below example utilizes the h2o package, and requires Java to be installed on your machine.
* install Java: https://www.java.com/en/download/help/mac_install.html
* h2o SuperLearner example: https://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/stacked-ensembles.html

Check out some other tutorials: 
* Python mlens library: https://mlens.readthedocs.io/en/0.1.x/install/
* Machine Learning Mastery: https://machinelearningmastery.com/super-learner-ensemble-in-python/
* KDNuggets: https://www.kdnuggets.com/2018/02/introduction-python-ensembles.html/2#comments

The quintessential R guide: 
* Guide to SuperLearner: https://cran.r-project.org/web/packages/SuperLearner/vignettes/Guide-to-SuperLearner.html

Read the papers: 
* [Van der Laan, M.J.; Polley, E.C.; Hubbard, A.E. Super Learner. Stat. Appl. Genet. Mol. Biol. 2007, 6, 1–21.](https://www.degruyter.com/document/doi/10.2202/1544-6115.1309/html)
* [Polley, E.C.; van der Laan, M.J. Super Learner in Prediction, UC Berkeley Division of Biostatistics Working Paper Series Paper 266.](https://biostats.bepress.com/ucbbiostat/paper266)

## H2O SuperLearner ensemble

Use machine learning ensembles to detect whether or not (simulated) particle collisions produce the Higgs Boson particle or not. Learn more about the data: https://www.kaggle.com/c/higgs/overview

## What is the Higgs Boson?

"The Higgs boson is the fundamental particle associated with the Higgs field, a field that gives mass to other fundamental particles such as electrons and quarks. A particle’s mass determines how much it resists changing its speed or position when it encounters a force. Not all fundamental particles have mass. The photon, which is the particle of light and carries the electromagnetic force, has no mass at all." (https://www.energy.gov/science/doe-explainsthe-higgs-boson)


![hb](img/hb.png)

### Install h2o and Java

In [1]:
# !pip install h2o

# Requires install of Java
# https://www.java.com/en/download/help/mac_install.html

### Import

In [2]:
import h2o
from h2o.estimators.random_forest import H2ORandomForestEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.estimators.stackedensemble import H2OStackedEnsembleEstimator
from h2o.grid.grid_search import H2OGridSearch
from __future__ import print_function

# turn off progress bars
h2o.no_progress()

### Initialize an h2o cluster

In [3]:
h2o.init(nthreads=-1, max_mem_size='2G')

Checking whether there is an H2O instance running at http://localhost:54321 .

.

.

.

.

 not found.
Attempting to start a local H2O server...


  Java Version: java version "1.8.0_321"; Java(TM) SE Runtime Environment (build 1.8.0_321-b07); Java HotSpot(TM) 64-Bit Server VM (build 25.321-b07, mixed mode)
  Starting server from /Users/evanmuzzall/opt/anaconda3/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/9g/fhnd1v790cj5ccxlv4rcvsy40000gq/T/tmphwdpzlj3
  JVM stdout: /var/folders/9g/fhnd1v790cj5ccxlv4rcvsy40000gq/T/tmphwdpzlj3/h2o_evanmuzzall_started_from_python.out
  JVM stderr: /var/folders/9g/fhnd1v790cj5ccxlv4rcvsy40000gq/T/tmphwdpzlj3/h2o_evanmuzzall_started_from_python.err


  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ...

 successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,America/Los_Angeles
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.34.0.1
H2O_cluster_version_age:,5 months and 15 days !!!
H2O_cluster_name:,H2O_from_python_evanmuzzall_3eh5z8
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.778 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


### Import a sample binary outcome train/test set into H2O

In [4]:
train = h2o.import_file("https://s3.amazonaws.com/erin-data/higgs/higgs_train_10k.csv")
test = h2o.import_file("https://s3.amazonaws.com/erin-data/higgs/higgs_test_5k.csv")

In [5]:
train

response,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28
1,0.869293,-0.635082,0.22569,0.32747,-0.689993,0.754202,-0.248573,-1.09206,0,1.37499,-0.653674,0.930349,1.10744,1.1389,-1.5782,-1.04699,0,0.65793,-0.0104546,-0.0457672,3.10196,1.35376,0.979563,0.978076,0.920005,0.721657,0.988751,0.876678
1,0.907542,0.329147,0.359412,1.49797,-0.31301,1.09553,-0.557525,-1.58823,2.17308,0.812581,-0.213642,1.27101,2.21487,0.499994,-1.26143,0.732156,0,0.398701,-1.13893,-0.00081911,0,0.30222,0.833048,0.9857,0.978098,0.779732,0.992356,0.798343
1,0.798835,1.47064,-1.63597,0.453773,0.425629,1.10487,1.28232,1.38166,0,0.851737,1.54066,-0.81969,2.21487,0.99349,0.35608,-0.208778,2.54822,1.25695,1.12885,0.900461,0,0.909753,1.10833,0.985692,0.951331,0.803252,0.865924,0.780118
0,1.34438,-0.876626,0.935913,1.99205,0.882454,1.78607,-1.64678,-0.942383,0,2.42326,-0.676016,0.736159,2.21487,1.29872,-1.43074,-0.364658,0,0.745313,-0.678379,-1.36036,0,0.946652,1.0287,0.998656,0.728281,0.8692,1.02674,0.957904
1,1.10501,0.321356,1.5224,0.882808,-1.20535,0.681466,-1.07046,-0.921871,0,0.800872,1.02097,0.971407,2.21487,0.596761,-0.350273,0.631194,0,0.479999,-0.373566,0.113041,0,0.755856,1.36106,0.98661,0.838085,1.1333,0.872245,0.808487
0,1.59584,-0.607811,0.00707492,1.81845,-0.111906,0.84755,-0.566437,1.58124,2.17308,0.755421,0.64311,1.42637,0,0.921661,-1.19043,-1.61559,0,0.651114,-0.654227,-1.27434,3.10196,0.823761,0.938191,0.971758,0.789176,0.430553,0.961357,0.957818
1,0.409391,-1.88468,-1.02729,1.67245,-1.6046,1.33801,0.0554274,0.0134659,2.17308,0.509783,-1.03834,0.707862,0,0.746918,-0.358465,-1.64665,0,0.367058,0.0694965,1.37713,3.10196,0.869418,1.22208,1.00063,0.545045,0.698653,0.977314,0.828786
1,0.933895,0.62913,0.527535,0.238033,-0.966569,0.547811,-0.0594392,-1.70687,2.17308,0.941003,-2.65373,-0.15722,0,1.03037,-0.175505,0.523021,2.54822,1.37355,1.29125,-1.46745,0,0.901837,1.08367,0.979696,0.7833,0.849195,0.894356,0.774879
1,1.40514,0.536603,0.689554,1.17957,-0.110061,3.2024,-1.52696,-1.57603,0,2.93154,0.567342,-0.130033,2.21487,1.78712,0.899499,0.585151,2.54822,0.401865,-0.151202,1.16349,0,1.66707,4.03927,1.17583,1.04535,1.54297,3.53483,2.74075
1,1.17657,0.104161,1.397,0.479721,0.265513,1.13556,1.53483,-0.253291,0,1.02725,0.534316,1.18002,0,2.40566,0.0875568,-0.976534,2.54822,1.25038,0.268541,0.530334,0,0.833175,0.773968,0.98575,1.1037,0.84914,0.937104,0.812364


In [6]:
print(train.shape)
print(test.shape)

(10000, 29)
(5000, 29)


### Identify predictors and response

In [7]:
x = train.columns
y = "response"
x.remove(y)

### For binary classification, response should be a factor

In [8]:
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

### Number of CV folds (to generate level-one data for stacking)

In [9]:
nfolds = 5

### How to stack

There are a few ways to assemble a list of models to stack together:
1. Train individual models and put them in a list
2. Train a grid of models
3. Train several grids of models

>Note: All base models must have the same cross-validation folds and the cross-validated predicted values must be kept.

## 1. Generate a 2-model ensemble 

Use three algorithms: 
1. random forest
2. gradient boosted machine
3. lasso

TODO: add RF and GBM defining characteristics

TODO: show how changing hyperparamters randomly can lead to overfitting (specifically # trees)

### Train and cross-validate a random forest

In [10]:
rf = H2ORandomForestEstimator(ntrees = 100,
                              nfolds = nfolds,
                              fold_assignment = 'Modulo',
                              keep_cross_validation_predictions = True,
                              seed = 1)
rf.train(x = x, y = y, training_frame = train)

Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  DRF_model_python_1646239408642_1


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,100.0,100.0,1893920.0,20.0,20.0,20.0,1394.0,1632.0,1502.23




ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.19950487132058212
RMSE: 0.446659681771908
LogLoss: 0.5827049508751073
Mean Per-Class Error: 0.30843686874008425
AUC: 0.7598007062584857
AUCPR: 0.7763300405428175
Gini: 0.5196014125169715

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4104450374569011: 


,,0,1,Error,Rate
0,0,2150.0,2555.0,0.543,(2555.0/4705.0)
1,1,716.0,4579.0,0.1352,(716.0/5295.0)
2,Total,2866.0,7134.0,0.3271,(3271.0/10000.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...


Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error
0,,2022-03-02 08:44:10,20.953 sec,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,,2022-03-02 08:44:10,21.023 sec,1.0,0.642818,14.048395,0.577384,0.589593,1.144218,0.464917
2,,2022-03-02 08:44:10,21.062 sec,2.0,0.615404,11.895281,0.595767,0.598674,1.167648,0.470126
3,,2022-03-02 08:44:10,21.102 sec,3.0,0.596109,10.322973,0.606984,0.606263,1.188141,0.471944
4,,2022-03-02 08:44:10,21.139 sec,4.0,0.578952,8.963477,0.617051,0.613281,1.205225,0.473260
5,,2022-03-02 08:44:10,21.169 sec,5.0,0.566757,7.823122,0.621447,0.619131,1.222342,0.472650
6,,2022-03-02 08:44:10,21.202 sec,6.0,0.555494,6.782681,0.625539,0.623303,1.235544,0.472576
7,,2022-03-02 08:44:10,21.236 sec,7.0,0.542668,5.762143,0.634190,0.632386,1.256193,0.470778
8,,2022-03-02 08:44:10,21.269 sec,8.0,0.534223,5.001247,0.637938,0.634830,1.257749,0.470999
9,,2022-03-02 08:44:10,21.304 sec,9.0,0.525930,4.327270,0.643458,0.638581,1.259426,0.416980



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,x26,15628.322266,1.000000,0.101301
1,x28,9415.993164,0.602495,0.061034
2,x27,9318.841797,0.596279,0.060404
3,x6,7188.417969,0.459961,0.046595
4,x25,7121.043945,0.455650,0.046158
5,x23,7112.994141,0.455135,0.046106
6,x4,6268.571289,0.401103,0.040632
7,x1,5769.832031,0.369191,0.037400
8,x10,5658.928223,0.362094,0.036681
9,x2,5388.754395,0.344807,0.034929



See the whole table with table.as_data_frame()


### Random forest test set performance

In [11]:
rf.model_performance(test)


ModelMetricsBinomial: drf
** Reported on test data. **

MSE: 0.19456237288967487
RMSE: 0.4410922498635346
LogLoss: 0.5723749503451226
Mean Per-Class Error: 0.2902132075244037
AUC: 0.7786910723119803
AUCPR: 0.8019489749908004
Gini: 0.5573821446239606

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3900028567254543: 


,,0,1,Error,Rate
0,0,957.0,1358.0,0.5866,(1358.0/2315.0)
1,1,280.0,2405.0,0.1043,(280.0/2685.0)
2,Total,1237.0,3763.0,0.3276,(1638.0/5000.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.390003,0.745968,280.0
1,max f2,0.252355,0.861172,348.0
2,max f0point5,0.545964,0.738321,184.0
3,max accuracy,0.503137,0.710400,211.0
4,max precision,0.976952,1.000000,0.0
5,max recall,0.145391,1.000000,382.0
6,max specificity,0.976952,1.000000,0.0
7,max absolute_mcc,0.515110,0.418565,204.0
8,max min_per_class_accuracy,0.513699,0.709125,205.0
9,max mean_per_class_accuracy,0.515110,0.709787,204.0



Gains/Lift Table: Avg response rate: 53.70 %, avg score: 52.30 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.0100,0.892945,1.824953,1.824953,0.980000,0.916686,0.980000,0.916686,0.018250,0.018250,82.495345,82.495345,0.017818
1,2,0.0200,0.869782,1.713222,1.769088,0.920000,0.881167,0.950000,0.898926,0.017132,0.035382,71.322160,76.908752,0.033222
2,3,0.0300,0.854788,1.713222,1.750466,0.920000,0.862835,0.940000,0.886896,0.017132,0.052514,71.322160,75.046555,0.048626
3,4,0.0400,0.840256,1.862197,1.778399,1.000000,0.847272,0.955000,0.876990,0.018622,0.071136,86.219739,77.839851,0.067248
4,5,0.0500,0.827591,1.787709,1.780261,0.960000,0.833668,0.956000,0.868325,0.017877,0.089013,78.770950,78.026071,0.084261
5,6,0.1000,0.771550,1.623836,1.702048,0.872000,0.797458,0.914000,0.832892,0.081192,0.170205,62.383613,70.204842,0.151630
6,7,0.1500,0.726679,1.646182,1.683426,0.884000,0.748252,0.904000,0.804678,0.082309,0.252514,64.618250,68.342644,0.221412
7,8,0.2012,0.690000,1.498487,1.636364,0.804688,0.706711,0.878728,0.779748,0.076723,0.329236,49.848696,63.636431,0.276537
8,9,0.3000,0.628621,1.368376,1.548107,0.734818,0.658471,0.831333,0.739808,0.135196,0.464432,36.837582,54.810677,0.355145
9,10,0.4000,0.572950,1.180633,1.456238,0.634000,0.600572,0.782000,0.704999,0.118063,0.582495,18.063315,45.623836,0.394158


### Train and cross-validate a gradient boosted machine

In [12]:
gbm = H2OGradientBoostingEstimator(distribution = "bernoulli",
                                   ntrees = 10,
                                   max_depth = 3,
                                   min_rows = 2,
                                   learn_rate = 0.2,
                                   nfolds = nfolds,
                                   fold_assignment = "Modulo",
                                   keep_cross_validation_predictions = True,
                                   seed = 1)
gbm.train(x = x, y = y, training_frame = train)

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_model_python_1646239408642_1062


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,10.0,10.0,1580.0,3.0,3.0,3.0,8.0,8.0,8.0




ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.20052170746266884
RMSE: 0.44779650228945383
LogLoss: 0.5879177464092424
Mean Per-Class Error: 0.29613223631461116
AUC: 0.7735466157694937
AUCPR: 0.7909110775032231
Gini: 0.5470932315389874

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4424199442800167: 


,,0,1,Error,Rate
0,0,2433.0,2272.0,0.4829,(2272.0/4705.0)
1,1,818.0,4477.0,0.1545,(818.0/5295.0)
2,Total,3251.0,6749.0,0.309,(3090.0/10000.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...


Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error
0,,2022-03-02 08:44:15,1.059 sec,0.0,0.499129,0.691406,0.500000,0.529500,1.000000,0.4705
1,,2022-03-02 08:44:16,1.074 sec,1.0,0.488236,0.669779,0.689191,0.674970,1.340835,0.3927
2,,2022-03-02 08:44:16,1.087 sec,2.0,0.479807,0.653146,0.719111,0.725388,1.545030,0.3958
3,,2022-03-02 08:44:16,1.101 sec,3.0,0.473698,0.641037,0.724650,0.731284,1.569032,0.3933
4,,2022-03-02 08:44:16,1.121 sec,4.0,0.467046,0.627816,0.746638,0.760063,1.683844,0.3692
5,,2022-03-02 08:44:16,1.136 sec,5.0,0.462994,0.619629,0.750899,0.766012,1.696473,0.3404
6,,2022-03-02 08:44:16,1.151 sec,6.0,0.458301,0.609973,0.756603,0.772386,1.733566,0.3515
7,,2022-03-02 08:44:16,1.165 sec,7.0,0.455569,0.604192,0.758101,0.773795,1.748075,0.3512
8,,2022-03-02 08:44:16,1.179 sec,8.0,0.452777,0.598335,0.763715,0.778986,1.754413,0.3180
9,,2022-03-02 08:44:16,1.195 sec,9.0,0.449743,0.591929,0.768491,0.784821,1.776659,0.3207



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,x26,565.332336,1.000000,0.410859
1,x28,204.503998,0.361741,0.148625
2,x27,189.230789,0.334725,0.137525
3,x23,124.527618,0.220273,0.090501
4,x6,121.551117,0.215008,0.088338
5,x25,99.598267,0.176176,0.072384
6,x4,44.359753,0.078467,0.032239
7,x10,18.008587,0.031855,0.013088
8,x22,3.660778,0.006475,0.002660
9,x18,3.097153,0.005478,0.002251



See the whole table with table.as_data_frame()


### Gradient boosted machine test set performance

In [13]:
gbm.model_performance(test)


ModelMetricsBinomial: gbm
** Reported on test data. **

MSE: 0.20579980230216896
RMSE: 0.453651630992515
LogLoss: 0.5991607617229083
Mean Per-Class Error: 0.3132779098342524
AUC: 0.7522686229794354
AUCPR: 0.7766705936284869
Gini: 0.5045372459588708

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.42008076282776335: 


,,0,1,Error,Rate
0,0,948.0,1367.0,0.5905,(1367.0/2315.0)
1,1,325.0,2360.0,0.121,(325.0/2685.0)
2,Total,1273.0,3727.0,0.3384,(1692.0/5000.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.420081,0.736120,265.0
1,max f2,0.271036,0.859410,354.0
2,max f0point5,0.544346,0.716075,160.0
3,max accuracy,0.484325,0.682800,209.0
4,max precision,0.788080,0.977778,4.0
5,max recall,0.143335,1.000000,396.0
6,max specificity,0.803916,0.999568,0.0
7,max absolute_mcc,0.544346,0.374728,160.0
8,max min_per_class_accuracy,0.506104,0.676890,192.0
9,max mean_per_class_accuracy,0.544346,0.686722,160.0



Gains/Lift Table: Avg response rate: 53.70 %, avg score: 52.40 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.0178,0.788097,1.820350,1.820350,0.977528,0.793217,0.977528,0.793217,0.032402,0.032402,82.035026,82.035026,0.031538
1,2,0.0316,0.779039,1.700267,1.767909,0.913043,0.779782,0.949367,0.787350,0.023464,0.055866,70.026718,76.790892,0.052410
2,3,0.0406,0.771638,1.779433,1.770464,0.955556,0.776606,0.950739,0.784968,0.016015,0.071881,77.943306,77.046353,0.067561
3,4,0.0532,0.766379,1.507493,1.708181,0.809524,0.767939,0.917293,0.780935,0.018994,0.090875,50.749313,70.818107,0.081372
4,5,0.1000,0.728488,1.639370,1.675978,0.880342,0.745772,0.900000,0.764479,0.076723,0.167598,63.937035,67.597765,0.145999
5,6,0.1502,0.692784,1.468984,1.606796,0.788845,0.713905,0.862850,0.747576,0.073743,0.241341,46.898440,60.679615,0.196848
6,7,0.2014,0.670097,1.513035,1.582960,0.812500,0.676769,0.850050,0.729575,0.077467,0.318808,51.303538,58.296025,0.253581
7,8,0.3022,0.624524,1.296887,1.487540,0.696429,0.647864,0.798809,0.702320,0.130726,0.449534,29.688747,48.753955,0.318217
8,9,0.4002,0.571753,1.189526,1.414563,0.638776,0.598000,0.759620,0.676774,0.116574,0.566108,18.952609,41.456274,0.358333
9,10,0.5002,0.508286,1.027933,1.337268,0.552000,0.540824,0.718113,0.649595,0.102793,0.668901,2.793296,33.726770,0.364366


## 3. Train a stacked ensemble using the GBM and RF above

What's going on here - anything suspicious?

In [14]:
ensemble = H2OStackedEnsembleEstimator(model_id = "my_ensemble_binomial",
                                       base_models = [rf, gbm])
ensemble.train(x = x, y = y, training_frame = train)

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  my_ensemble_binomial

No model summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.034414533979938884
RMSE: 0.1855115467563647
LogLoss: 0.18617000727747787
Null degrees of freedom: 9999
Residual degrees of freedom: 9997
Null deviance: 13828.113387424342
Residual deviance: 3723.4001455495572
AIC: 3729.4001455495572
AUC: 0.9999900453478558
AUCPR: 0.9999911890971335
Gini: 0.9999800906957117

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4952892042214926: 


,,0,1,Error,Rate
0,0,4692.0,13.0,0.0028,(13.0/4705.0)
1,1,3.0,5292.0,0.0006,(3.0/5295.0)
2,Total,4695.0,5305.0,0.0016,(16.0/10000.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.495289,0.998491,200.0
1,max f2,0.495289,0.999056,200.0
2,max f0point5,0.550520,0.998978,192.0
3,max accuracy,0.512894,0.998400,198.0
4,max precision,0.947625,1.000000,0.0
5,max recall,0.432910,1.000000,211.0
6,max specificity,0.947625,1.000000,0.0
7,max absolute_mcc,0.495289,0.996790,200.0
8,max min_per_class_accuracy,0.512894,0.998300,198.0
9,max mean_per_class_accuracy,0.512894,0.998394,198.0



Gains/Lift Table: Avg response rate: 52.95 %, avg score: 52.69 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.01,0.939745,1.888574,1.888574,1.000,0.943046,1.000000,0.943046,0.018886,0.018886,88.857413,88.857413,0.018886
1,2,0.02,0.935499,1.888574,1.888574,1.000,0.937644,1.000000,0.940345,0.018886,0.037771,88.857413,88.857413,0.037771
2,3,0.03,0.932326,1.888574,1.888574,1.000,0.933926,1.000000,0.938205,0.018886,0.056657,88.857413,88.857413,0.056657
3,4,0.04,0.929278,1.888574,1.888574,1.000,0.930792,1.000000,0.936352,0.018886,0.075543,88.857413,88.857413,0.075543
4,5,0.05,0.926211,1.888574,1.888574,1.000,0.927677,1.000000,0.934617,0.018886,0.094429,88.857413,88.857413,0.094429
5,6,0.10,0.912796,1.888574,1.888574,1.000,0.919384,1.000000,0.927000,0.094429,0.188857,88.857413,88.857413,0.188857
6,7,0.15,0.899271,1.888574,1.888574,1.000,0.905914,1.000000,0.919971,0.094429,0.283286,88.857413,88.857413,0.283286
7,8,0.20,0.883838,1.888574,1.888574,1.000,0.891593,1.000000,0.912877,0.094429,0.377715,88.857413,88.857413,0.377715
8,9,0.30,0.849319,1.888574,1.888574,1.000,0.867316,1.000000,0.897690,0.188857,0.566572,88.857413,88.857413,0.566572
9,10,0.40,0.794174,1.888574,1.888574,1.000,0.824146,1.000000,0.879304,0.188857,0.755430,88.857413,88.857413,0.755430


### Ensemble performance on test set

In [15]:
perf_stack_test = ensemble.model_performance(test)
perf_stack_test


ModelMetricsBinomialGLM: stackedensemble
** Reported on test data. **

MSE: 0.19097357833656903
RMSE: 0.43700523833996435
LogLoss: 0.5620616630522318
Null degrees of freedom: 4999
Residual degrees of freedom: 4997
Null deviance: 6905.196403260959
Residual deviance: 5620.616630522318
AIC: 5626.616630522318
AUC: 0.7787452892036794
AUCPR: 0.8019528045514408
Gini: 0.5574905784073587

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3261076325075436: 


,,0,1,Error,Rate
0,0,971.0,1344.0,0.5806,(1344.0/2315.0)
1,1,285.0,2400.0,0.1061,(285.0/2685.0)
2,Total,1256.0,3744.0,0.3258,(1629.0/5000.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.326108,0.746617,288.0
1,max f2,0.150854,0.860876,365.0
2,max f0point5,0.609668,0.740688,155.0
3,max accuracy,0.486185,0.708400,212.0
4,max precision,0.942076,1.000000,0.0
5,max recall,0.087693,1.000000,389.0
6,max specificity,0.942076,1.000000,0.0
7,max absolute_mcc,0.543978,0.420678,186.0
8,max min_per_class_accuracy,0.502175,0.704968,204.0
9,max mean_per_class_accuracy,0.543978,0.710596,186.0



Gains/Lift Table: Avg response rate: 53.70 %, avg score: 52.14 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.01,0.916338,1.862197,1.862197,1.000,0.924969,1.000000,0.924969,0.018622,0.018622,86.219739,86.219739,0.018622
1,2,0.02,0.909256,1.713222,1.787709,0.920,0.912814,0.960000,0.918892,0.017132,0.035754,71.322160,78.770950,0.034026
2,3,0.03,0.900496,1.713222,1.762880,0.920,0.905124,0.946667,0.914302,0.017132,0.052886,71.322160,76.288020,0.049431
3,4,0.04,0.893308,1.824953,1.778399,0.980,0.896996,0.955000,0.909976,0.018250,0.071136,82.495345,77.839851,0.067248
4,5,0.05,0.884151,1.750466,1.772812,0.940,0.889545,0.952000,0.905890,0.017505,0.088641,75.046555,77.281192,0.083457
5,6,0.10,0.844421,1.653631,1.713222,0.888,0.864111,0.920000,0.885000,0.082682,0.171322,65.363128,71.322160,0.154044
6,7,0.15,0.803300,1.608939,1.678461,0.864,0.824145,0.901333,0.864715,0.080447,0.251769,60.893855,67.846058,0.219804
7,8,0.20,0.768014,1.482309,1.629423,0.796,0.786533,0.875000,0.845170,0.074115,0.325885,48.230912,62.942272,0.271889
8,9,0.30,0.690902,1.337058,1.531968,0.718,0.730971,0.822667,0.807103,0.133706,0.459590,33.705773,53.196772,0.344688
9,10,0.40,0.605813,1.284916,1.470205,0.690,0.647687,0.789500,0.767249,0.128492,0.588082,28.491620,47.020484,0.406224


### Compare to base learner performance on the test set

The ensemble is a little better, but it is still pretty close...

In [16]:
perf_gbm_test = gbm.model_performance(test)
perf_rf_test = rf.model_performance(test)
baselearner_best_auc_test = max(perf_gbm_test.auc(), perf_rf_test.auc())
stack_auc_test = perf_stack_test.auc()
print("Best Base-learner Test AUC:  {0}".format(baselearner_best_auc_test))
print("Ensemble Test AUC:  {0}".format(stack_auc_test))

Best Base-learner Test AUC:  0.7786910723119803
Ensemble Test AUC:  0.7787452892036794


### Generate predictions on a test set (if neccessary)

In [17]:
predictions = ensemble.predict(test)
predictions

predict,p0,p1
0,0.70526,0.29474
1,0.513641,0.486359
1,0.612075,0.387925
1,0.247459,0.752541
1,0.496399,0.503601
1,0.297365,0.702635
1,0.341914,0.658086
0,0.71747,0.28253
1,0.634757,0.365243
0,0.708404,0.291596


## 4. Generate a random grid of models and stack them together

### Specify GBM hyperparameters for the grid

Keep in mind it might be easier to define sequences of numbers for your various hyperparameter tunings. 

Also, exponential and logarithmic scales are probably preferred to linear ones.

In [18]:
hyper_params = {"learn_rate": [0.01, 0.03, 0.05, 0.2, 0.3, 0.4, 0.7, 0.8],
                "max_depth": [3, 4, 5, 6, 9],
                "sample_rate": [0.7, 0.8, 0.9, 1.0],
                "col_sample_rate": [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]}
search_criteria = {"strategy": "RandomDiscrete", "max_models": 3, "seed": 1}

In [19]:
# Train the grid
grid = H2OGridSearch(model=H2OGradientBoostingEstimator(ntrees = 10,
                                                        seed = 1,
                                                        nfolds = nfolds,
                                                        fold_assignment = 'Modulo',
                                                        keep_cross_validation_predictions = True),
                     hyper_params=hyper_params,
                     search_criteria=search_criteria,
                     grid_id="gbm_grid_binomial")
grid.train(x=x, y=y, training_frame=train)

    col_sample_rate learn_rate max_depth sample_rate  \
0               0.4        0.8         3         0.7   
1               0.2        0.8         4         0.8   
2               0.7       0.03         5         0.9   

                   model_ids             logloss  
0  gbm_grid_binomial_model_3     0.5866851338931  
1  gbm_grid_binomial_model_2  0.6144661087268164  
2  gbm_grid_binomial_model_1  0.6535517466453371  


## 5. Train a stacked ensemble using the GBM grid

In [20]:
ensemble = H2OStackedEnsembleEstimator(model_id = "my_ensemble_gbm_grid_binomial",
                                       base_models = grid.model_ids)
ensemble.train(x = x, y = y, training_frame = train)

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  my_ensemble_gbm_grid_binomial

No model summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.17897505903570504
RMSE: 0.42305443980143387
LogLoss: 0.5357403550336852
Null degrees of freedom: 9999
Residual degrees of freedom: 9996
Null deviance: 13828.113387424342
Residual deviance: 10714.807100673706
AIC: 10722.807100673706
AUC: 0.8093464349400262
AUCPR: 0.8219046127206588
Gini: 0.6186928698800525

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4037334009318257: 


,,0,1,Error,Rate
0,0,2723.0,1982.0,0.4213,(1982.0/4705.0)
1,1,779.0,4516.0,0.1471,(779.0/5295.0)
2,Total,3502.0,6498.0,0.2761,(2761.0/10000.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.403733,0.765878,248.0
1,max f2,0.241819,0.864462,330.0
2,max f0point5,0.579602,0.756389,167.0
3,max accuracy,0.515885,0.734700,195.0
4,max precision,0.904517,1.000000,0.0
5,max recall,0.118691,1.000000,385.0
6,max specificity,0.904517,1.000000,0.0
7,max absolute_mcc,0.515885,0.467751,195.0
8,max min_per_class_accuracy,0.528054,0.731350,190.0
9,max mean_per_class_accuracy,0.515885,0.733957,195.0



Gains/Lift Table: Avg response rate: 52.95 %, avg score: 52.96 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.01,0.887257,1.869688,1.869688,0.990,0.893840,0.990000,0.893840,0.018697,0.018697,86.968839,86.968839,0.018484
1,2,0.02,0.879296,1.794145,1.831917,0.950,0.883362,0.970000,0.888601,0.017941,0.036638,79.414542,83.191690,0.035363
2,3,0.03,0.872364,1.850803,1.838212,0.980,0.875594,0.973333,0.884265,0.018508,0.055146,85.080264,83.821215,0.053446
3,4,0.04,0.866487,1.775260,1.822474,0.940,0.869257,0.965000,0.880513,0.017753,0.072899,77.525968,82.247403,0.069923
4,5,0.05,0.860642,1.756374,1.809254,0.930,0.863476,0.958000,0.877106,0.017564,0.090463,75.637394,80.925401,0.085999
5,6,0.10,0.834422,1.688385,1.748820,0.894,0.847602,0.926000,0.862354,0.084419,0.174882,68.838527,74.881964,0.159154
6,7,0.15,0.807362,1.650614,1.716084,0.874,0.821115,0.908667,0.848607,0.082531,0.257413,65.061379,71.608436,0.228295
7,8,0.20,0.776635,1.578848,1.681775,0.836,0.792606,0.890500,0.834607,0.078942,0.336355,57.884797,68.177526,0.289809
8,9,0.30,0.711611,1.423985,1.595845,0.754,0.744221,0.845000,0.804478,0.142398,0.478754,42.398489,59.584514,0.379923
9,10,0.40,0.629691,1.278565,1.516525,0.677,0.671084,0.803000,0.771130,0.127856,0.606610,27.856468,51.652502,0.439129


### Eval ensemble performance on the test data

In [21]:
perf_stack_test = ensemble.model_performance(test)
perf_stack_test


ModelMetricsBinomialGLM: stackedensemble
** Reported on test data. **

MSE: 0.19646981512542924
RMSE: 0.44324915693707667
LogLoss: 0.5753535672126392
Null degrees of freedom: 4999
Residual degrees of freedom: 4996
Null deviance: 6905.196403260959
Residual deviance: 5753.5356721263925
AIC: 5761.5356721263925
AUC: 0.7658850264045916
AUCPR: 0.7928758222962261
Gini: 0.5317700528091831

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.35412909231741746: 


,,0,1,Error,Rate
0,0,1068.0,1247.0,0.5387,(1247.0/2315.0)
1,1,367.0,2318.0,0.1367,(367.0/2685.0)
2,Total,1435.0,3565.0,0.3228,(1614.0/5000.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.354129,0.741760,275.0
1,max f2,0.178761,0.860706,362.0
2,max f0point5,0.580789,0.727106,159.0
3,max accuracy,0.520792,0.696400,190.0
4,max precision,0.897976,1.000000,0.0
5,max recall,0.094361,1.000000,396.0
6,max specificity,0.897976,1.000000,0.0
7,max absolute_mcc,0.580789,0.394906,159.0
8,max min_per_class_accuracy,0.519258,0.695464,191.0
9,max mean_per_class_accuracy,0.579075,0.696613,160.0



Gains/Lift Table: Avg response rate: 53.70 %, avg score: 52.42 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.01,0.887794,1.862197,1.862197,1.000,0.891943,1.000000,0.891943,0.018622,0.018622,86.219739,86.219739,0.018622
1,2,0.02,0.878222,1.787709,1.824953,0.960,0.883015,0.980000,0.887479,0.017877,0.036499,78.770950,82.495345,0.035635
2,3,0.03,0.872278,1.824953,1.824953,0.980,0.875645,0.980000,0.883535,0.018250,0.054749,82.495345,82.495345,0.053453
3,4,0.04,0.865552,1.713222,1.797020,0.920,0.869065,0.965000,0.879917,0.017132,0.071881,71.322160,79.702048,0.068857
4,5,0.05,0.858329,1.750466,1.787709,0.940,0.861865,0.960000,0.876307,0.017505,0.089385,75.046555,78.770950,0.085066
5,6,0.10,0.830711,1.564246,1.675978,0.840,0.844201,0.900000,0.860254,0.078212,0.167598,56.424581,67.597765,0.145999
6,7,0.15,0.799415,1.579143,1.643700,0.848,0.815299,0.882667,0.845269,0.078957,0.246555,57.914339,64.369957,0.208542
7,8,0.20,0.768224,1.534451,1.616387,0.824,0.785330,0.868000,0.830284,0.076723,0.323277,53.445065,61.638734,0.266258
8,9,0.30,0.697137,1.333333,1.522036,0.716,0.733487,0.817333,0.798018,0.133333,0.456611,33.333333,52.203600,0.338252
9,10,0.40,0.614587,1.199255,1.441341,0.644,0.654905,0.774000,0.762240,0.119926,0.576536,19.925512,44.134078,0.381288


## 6. Compare to base learner performance on the test set

In [22]:
baselearner_best_auc_test = max([h2o.get_model(model).model_performance(test_data=test).auc() for model in grid.model_ids])
stack_auc_test = perf_stack_test.auc()
print("Best Base-learner Test AUC:  {0}".format(baselearner_best_auc_test))
print("Ensemble Test AUC:  {0}".format(stack_auc_test))

Best Base-learner Test AUC:  0.7535296885746348
Ensemble Test AUC:  0.7658850264045916


### Generate predictions on a test set (if neccessary)

In [23]:
predictions2 = ensemble.predict(test)
predictions2

predict,p0,p1
0,0.752499,0.247501
0,0.692124,0.307876
1,0.490744,0.509256
1,0.293398,0.706602
1,0.514172,0.485828
1,0.292388,0.707612
1,0.336833,0.663167
0,0.661777,0.338223
1,0.612586,0.387414
1,0.627887,0.372113


## Deep learning basics

Deep learning is a subfield of machine learning that uses a variety of multi-layered artificial neural networks to model datasets and predict outcomes. 

### The human brain model key terms

The idea was originally modelled on the human brain, which has ~100 billion neurons. 
* The **soma** (neuron body) holds the architecture for cell function and energy processing. 
* **Dendrites** receive information from other neurons and transfer it towards the soma. 
* **Axons** send information from the soma towards other dendrites/soma. 
* Dendrites and axons are connected by a **synapse**. 

### Information transfer in the human brain

1. An outbound neuron produces an electrical signal called a **spike** that travel's to the synapse, where chemicals called **neurotransmitters**. 
2. Receptors on the inbound neuron receive the neurotransmitter, to generate another electrical signal to send the original signal to the soma. 
3. Whether or not neurons are fired in simultaneously or in success depend on the strength/amount of the spikes. If a certain threshold is crossed, the next neuron will be activated. 

![neuron](img/neuron.png)

[Wikipedia](https://en.wikipedia.org/wiki/Neuron)

## Are deep neural networks really like the human brain? 

Deep artificial networks were originally modelled on the human brain, and many argue for and against their likeness. See for yourself by reading the below posts!

* [Neural Networks](https://medium.com/nerd-for-tech/neural-networks-68531432fb5)
* [Do neural networks really work like neurons?](https://medium.com/swlh/do-neural-networks-really-work-like-neurons-667859dbfb4f)
* [Neural Networks Do Not Work Like Human Brains – Let’s Debunk The Myth](https://analyticsindiamag.com/neural-networks-not-work-like-human-brains-lets-debunk-myth/)
* [Artificial neural networks are more similar to the brain than they get credit for](https://bdtechtalks.com/2020/06/22/direct-fit-artificial-neural-networks/)
* [Here’s Why We May Need to Rethink Artificial Neural Networks](https://towardsdatascience.com/heres-why-we-may-need-to-rethink-artificial-neural-networks-c7492f51b7bc)
* [Artificial Neural Nets Finally Yield Clues to How Brains Learn](https://www.quantamagazine.org/artificial-neural-nets-finally-yield-clues-to-how-brains-learn-20210218/)

## Why deep learning?

Deep learning is ideal for all data types, but especially text, image, video, and sound because deep representative networks store these data as large matrices. Also, error is recycled (backpropagated) to update the model weights and make better predictions during the next epoch. . 

To understand deep networks, let's start with a toy example of a single feed forward neural network - a perceptron.

Read Goodfellow et al's Deep Learning Book to learn more: https://www.deeplearningbook.org/

In [24]:
import pandas as pd

# generate toy dataset
example = {'x1': [1, 0, 1, 1, 0], 
           'x2': [1, 1, 1, 1, 0], 
           'xm': [1, 0, 1, 1, 0],
           'output': ['yes', 'no', 'yes', 'yes', 'no']
           }
example_df = pd.DataFrame(data = example)
example_df

,x1,x2,xm,output
0,1,1,1,yes
1,0,1,0,no
2,1,1,1,yes
3,1,1,1,yes
4,0,0,0,no


![perceptron](img/perceptron.png)

Perceptron figure modified from [Sebastian Raschka's Single-Layer Neural Networks and Gradient Descent](https://sebastianraschka.com/Articles/2015_singlelayer_neurons.html)

Perceptron key terms: 
* **Layer:** the neural architecture or the network typology of a deep learning model, usually divided into variations of input, hidden, preprocessing, encoder/decoder, and output. 
* **Inputs/Nodes:** features/covariates/predictors/independent variables (the columns of 1's and 0s from `example_df` above), but they could be words from a text or pixels from an image. 
* **Weights:** the learnable parameters of a model that connect the input layer to the output via the net input (summation) and activation functions. Weights are often randomly initialized.
* **Bias term:** A placeholder "1" assures that we do not receive only 0 predictions of our features are zero or close to 0. 
* **Net input function:** computes the weighted sum of the input layer. 
* **Activation function:** determines if a neuron should be fired or not. In binary classification for example, this defines a threshold (0.5 for example) for determining if a 1 or 0 should be predicted. 
* **Output:** a node that contains the y prediction.
* **Error:** how far off an output prediction was. The weights are updated by adjusting the learning rate based on the error to reduce it for the next epoch. 
* **Epoch:** full pass of the training data. 
* **Backpropagation:** 
* **Hyperparameters:** our definition of the neural architecture, including but not limited to: number of hidden units, weight initialization, learning rate, batch size, dropout, etc.

## What makes a network "deep"?

A "deep" network is just network with multiple/many hidden layers for handling potential nonlinear transformations.

* Fully connected layer: a layer where all nodes are connected to every node in the next layer (as indicated by the purple arrows 

![deep](img/deep.png)

Example of "deep" network with two hidden layers modified from [DevSkrol's Artificial Neural Network Explained with an Regression Example](https://devskrol.com/2020/11/22/388/)

>NOTE: Bias term not shown for some reason?

## Classify images of cats and dogs

Let's go through François Chollet's "Image classification from scratch" [tutorial](https://keras.io/examples/vision/image_classification_from_scratch/) to examine this architecture and predict images of cats versus dogs. 

[Click here to open the Colab notebook](https://colab.research.google.com/github/keras-team/keras-io/blob/master/examples/vision/ipynb/image_classification_from_scratch.ipynb)

> NOTE: One pain point for working with your own images is importing them correctly. Schedule a consultation with SSDS if you need help! https://ssds.stanford.edu/

You should also check out his deep learning book Deep Learning with Python (R version also available): https://www.manning.com/books/deep-learning-with-python-second-edition

![dogcat](img/dogcat.jpg)